## 内存管理

### 栈内存管理

栈内存用于存储局部变量和函数参数。当函数被调用时，其参数和局部变量被“压入”栈中，当函数返回时，这些变量会被自动“弹出”栈并销毁。

自动内存分配：函数内部声明的变量自动分配在栈上。

优点：分配和释放速度快，无需手动管理。

（ 缺点：空间有限，大小在编译时确定。）

### 堆内存管理

堆内存用于动态内存分配，程序员可以在程序运行时请求特定大小的内存，使用完后需要手动释放。

#### malloc 与 free
在C++中，可以使用C语言中的malloc和free进行动态内存分配和释放。

malloc用于分配一定大小的内存，并返回指向该内存的指针。分配的内存未初始化。
free用于释放malloc分配的内存。

In [ ]:
int* ptr = (int*)malloc(sizeof(int) * 10); // 分配10个整数的空间
// 使用内存...
free(ptr); // 释放内存

#### new 与 delete

C++引入了new和delete操作符，提供了更符合面向对象的方式来处理动态内存。

new分配内存并调用构造函数（如果有的话），返回指向对象的指针。
delete释放内存并调用析构函数（如果有的话）。

In [ ]:
int* ptr = new int[10]; // 分配10个整数的空间
// 使用内存...
delete[] ptr; // 释放内存

In [ ]:
#include <iostream>

class MyClass {
public:
    MyClass() {
        std::cout << "Constructor called" << std::endl;
    }

    ~MyClass() {
        std::cout << "Destructor called" << std::endl;
    }
};

int main() {
    MyClass* myObject = new MyClass(); // 动态分配内存并构造对象
    // 执行一些操作...
    delete myObject; // 销毁对象并释放内存
    return 0;
}

### 智能指针

C++11引入了智能指针来简化内存管理。

std::unique_ptr：保证同一时间只有一个指针指向对象，当unique_ptr离开作用域时，它指向的对象会被自动删除。
std::shared_ptr：允许多个指针共享对同一个对象的所有权。当最后一个shared_ptr被销毁时，它指向的对象才会被删除。
std::weak_ptr：与std::shared_ptr配合使用，但不拥有对象。用于解决shared_ptr可能产生的循环引用问题。
使用智能指针可以大大减少内存泄漏的风险，是现代C++程序中推荐的内存管理方式。

In [ ]:
#include <iostream>
#include <memory>

class MyClass {
public:
    MyClass() {
        std::cout << "Constructor called" << std::endl;
    }

    ~MyClass() {
        std::cout << "Destructor called" << std::endl;
    }
};

int main() {
    std::unique_ptr<MyClass> myUniquePtr(new MyClass()); // 使用 unique_ptr 管理 MyClass 对象
    // 执行一些操作...
    // 当 unique_ptr 离开作用域时，MyClass 的实例会被自动销毁
    return 0;
}

std::shared_ptr 的 use_count() 方法返回指向当前对象的 shared_ptr 实例的数量。当这个计数降到0时，对象会被自动销毁。

In [ ]:
int main() {
    std::shared_ptr<MyClass> mySharedPtr1(new MyClass());
    std::shared_ptr<MyClass> mySharedPtr2 = mySharedPtr1;

    std::cout << "Shared pointers count: " << mySharedPtr1.use_count() << std::endl;
    mySharedPtr2.reset(); // 显式销毁 mySharedPtr2
    std::cout << "Shared pointers count after destroying mySharedPtr2: " << mySharedPtr1.use_count() << std::endl;
    mySharedPtr1.reset(); // 显式销毁 mySharedPtr1
    // 此时 MyClass 实例会被销毁
    return 0;
}

### std::weak_ptr 和循环引用问题

std::weak_ptr 是一种特殊的智能指针，它不拥有对象的所有权，因此不会影响对象的引用计数。weak_ptr 主要用于解决 shared_ptr 可能导致的循环引用问题。

循环引用通常发生在两个或多个 shared_ptr 实例互相持有对方的引用，导致它们的引用计数永远不会降到0，从而造成内存泄漏。

In [ ]:
//循环引用的例子
class B; // 前向声明

class A {
public:
    std::shared_ptr<B> b_ptr;
    ~A() {
        std::cout << "A destroyed" << std::endl;
    }
};

class B {
public:
    std::shared_ptr<A> a_ptr;
    ~B() {
        std::cout << "B destroyed" << std::endl;
    }
};

int main() {
    auto a = std::make_shared<A>();
    auto b = std::make_shared<B>();
    a->b_ptr = b;
    b->a_ptr = a;
    // 循环引用，A 和 B 不会被销毁
    return 0;
}


In [ ]:
/*使用 std::weak_ptr 可以解决循环引用问题。在上述示例中，如果我们将其中一个类中的 shared_ptr 改为 weak_ptr，循环引用就会被打破：
在这个修改后的示例中，A 类的 b_ptr 成员被改为 std::weak_ptr。这样，即使 a 和 b 互相引用，B 对象的引用计数也不会因为 A 对象的存在而永远大于0，从而可以被正确销毁。*/
class B;

class A {
public:
    std::weak_ptr<B> b_ptr; // 使用 weak_ptr
    ~A() {
        std::cout << "A destroyed" << std::endl;
    }
};

// B 类定义保持不变...

int main() {
    auto a = std::make_shared<A>();
    auto b = std::make_shared<B>();
    a->b_ptr = b;
    b->a_ptr = a;
    // 不再有循环引用，A 和 B 可以被正确销毁
    return 0;
}
